In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [4]:
!ls data

data_sets.json    doc_sim.csv       network_json.json


In [172]:
allDatasets = pd.read_json('data/data_sets.json')
print(allDatasets.shape)
allDatasets.head()

(10348, 14)


,additional_keywords,citation,coverages,data_set_id,date,description,family_identifier,identifier_list,mention_list,methodology,name,subjects,title,unique_identifier
0,ICPSR,,,1,2016-09-20 00:00:00+00:00,This study is part of a time-series collection...,,"[{'name': 'ICPSR data ID (dataId)', 'identifie...","[ANES study, ICPSR, SRC data, Surveys conducte...",,ANES 1952 Time Series Study,"candidates,congressional elections,domestic po...",ANES 1952 Time Series Study,10.3886/ICPSR07213
1,ICPSR,,,2,2016-09-22 00:00:00+00:00,This study is part of a time-series collection...,,"[{'name': 'ICPSR data ID (dataId)', 'identifie...","[American national election studies, SRC-CPS c...",,ANES 1956 Time Series Study,"candidates,congressional elections,domestic po...",ANES 1956 Time Series Study,10.3886/ICPSR07214
2,ICPSR,,,3,2016-09-22 00:00:00+00:00,This study is part of a time-series collection...,,"[{'name': 'ICPSR data ID (dataId)', 'identifie...",[],,ANES 1958 Time Series Study,"candidates,congressional elections,domestic po...",ANES 1958 Time Series Study,10.3886/ICPSR07215
3,ICPSR,,,4,2015-11-10 00:00:00+00:00,This study is part of a time-series collection...,,"[{'name': 'ICPSR data ID (dataId)', 'identifie...",[],,ANES 1960 Time Series Study,"candidates,congressional elections,domestic po...",ANES 1960 Time Series Study,10.3886/ICPSR07216
4,ICPSR,,,5,2016-12-01 00:00:00+00:00,This study is part of a time-series collection...,,"[{'name': 'ICPSR data ID (dataId)', 'identifie...",[],,ANES 1962 Time Series Study,"candidates,congressional elections,domestic po...",ANES 1962 Time Series Study,10.3886/ICPSR07217


In [173]:
import re

months = ['january',
             'feburary',
             'march',
             'april',
             'may',
             'june',
             'july',
             'august',
             'september',
             'october',
             'november',
             'december']

def removeMonth(name_str, months):
    '''
    Remove month strings in titles
    
    Return string (with white space)
    '''
    
    last_str = name_str.split(' ')[-1]
    if last_str == '':
        last_str = name_str.split(' ')[-2]
        n = 2
    else:
        n = 1
    if last_str in months:
        return ' '.join(name_str.split(' ')[:-n])
    else:
        return ' '.join(name_str.split(' '))

#Remove number and special characters
stripped_title_with_month = [re.sub(r"[^a-zA-Z]+", ' ', k).lower() for k in allDatasets.title]
#Remove months and white space
stripped_title = [re.sub(' ', '', removeMonth(k, months)) for k in stripped_title_with_month]

In [174]:
allDatasets['stripped_title'] = stripped_title
id_list = [[x] for x in allDatasets.data_set_id]
allDatasets['id_list'] = id_list

agg_func = {'id_list': 'sum'} #lambda x: np.concatenate(x)}

stripped_datasets = allDatasets.groupby(by = 'stripped_title').agg(agg_func)
print(stripped_datasets.shape)
stripped_datasets.head()

(5939, 1)


,id_list
stripped_title,
abcnewsabortioncubapoll,[2084]
abcnewsabuseandexcusepollfebruary,[4471]
abcnewsaidspoll,"[4629, 6519]"
abcnewsangryvoterpoll,[6903]
abcnewsanniversarypoll,[3822]


In [178]:
# Merge metadata back to the stacked dataframe

stripped_datasets['reference_id'] = [x[0] for x in stripped_datasets.id_list]

datasets_stacked = stripped_datasets.merge(allDatasets, left_on = 'reference_id', right_on = 'data_set_id', how = 'left')
datasets_stacked.head()

,id_list_x,reference_id,additional_keywords,citation,coverages,data_set_id,date,description,family_identifier,identifier_list,mention_list,methodology,name,subjects,title,unique_identifier,stripped_title,id_list_y
0,[2084],2084,,,,2084,2006-11-13 00:00:00+00:00,This special topic poll sought respondents' vi...,,"[{'name': 'ICPSR Data ID (dataId)', 'identifie...",[],,"ABC News Abortion/Cuba Poll, January 1998","abortion,Castro, Fidel,Clinton, Bill,diplomacy...","ABC News Abortion/Cuba Poll, January 1998",10.3886/ICPSR02506,abcnewsabortioncubapoll,[2084]
1,[4471],4471,,,,4471,1997-05-16 00:00:00+00:00,"For this special topic poll, respondents were ...",,"[{'name': 'ICPSR Data ID (dataId)', 'identifie...",[],,"ABC News Abuse and Excuse Poll, February 1994","assault,child abuse,Harding, Tonya,Kerrigan, N...","ABC News Abuse and Excuse Poll, February 1994",10.3886/ICPSR06619,abcnewsabuseandexcusepollfebruary,[4471]
2,"[4629, 6519]",4629,,,,4629,1998-05-20 00:00:00+00:00,This special topic poll sought respondents' vi...,,"[{'name': 'ICPSR Data ID (dataId)', 'identifie...",[],,"ABC News AIDS Poll, January 1996","AIDS,beliefs,health problems,HIV,Johnson, Magi...","ABC News AIDS Poll, January 1996",10.3886/ICPSR06833,abcnewsaidspoll,[4629]
3,[6903],6903,,,,6903,2007-04-02 00:00:00+00:00,In addition to providing an ongoing evaluation...,,"[{'name': 'ICPSR Data ID (dataId)', 'identifie...",[],,"ABC News Angry Voter Poll, April 1992","Bush, George H.W.,government performance,presi...","ABC News Angry Voter Poll, April 1992",10.3886/ICPSR09942,abcnewsangryvoterpoll,[6903]
4,[3822],3822,,,,3822,2008-01-24 00:00:00+00:00,"This special topic poll, conducted September 5...",,"[{'name': 'ICPSR Data ID (dataId)', 'identifie...",[],,"ABC News 9/11 Anniversary Poll, September 2006","anxiety,attitudes,Bush Administration (George ...","ABC News 9/11 Anniversary Poll, September 2006",10.3886/ICPSR04665,abcnewsanniversarypoll,[3822]


In [180]:
datasets_stacked.to_csv('datasets_stacked.csv')